# Thu thập dữ liệu

## Giới thiệu
Dự án xây dựng mô hình dự đoán giá ô tô thông qua dữ liệu thực tế lấy từ các website mua bán xe trực tuyến. Notebook này tóm tắt quy trình lấy dữ liệu từ hai trang mua bán lớn của Việt Nam và chuẩn hóa chúng cho việc mô hình hóa.

## Mục tiêu
- Thu thập các thông số kỹ thuật (features) chính xác từ mỗi website.
- Ghi nhận nguồn gốc dữ liệu để so sánh nhanh giữa hai trang.

## Phạm vi
- Nguồn: Chotot (API) và Bonbanh (web scraping).
- Dữ liệu: Tiêu đề, giá, thông số kỹ thuật, vị trí, người bán, số ảnh (nếu có).
- Kết quả: CSV trong `datasets/` phục vụ phân tích và huấn luyện mô hình.

In [ ]:
import requests 
from bs4 import BeautifulSoup  
import pandas as pd  
import re  

N = 100  # Số lượng mẫu cần thu thập từ mỗi trang web

## Phương pháp

### 1. Thu thập dữ liệu
- **Chotot**: Dùng API nội bộ để lấy ID và sinh URL rao bán xe, tránh phải parse toàn bộ trang tìm kiếm.
- **Bonbanh**: Đọc trang `bonbanh.com/oto` và scrape danh sách listing, sau đó truy cập từng URL chi tiết.

### 2. Trích xuất features
#### Chotot
- Tiêu đề/giá nằm ở các tag rõ ràng (`<h1>` và `<b class="p26z2wb">`).
- Tất cả features được gom trong `div.p1ja3eq0`; cặp `span.bwq0cbs` đầu là label, sau là giá trị.
- Đọc vị trí, người bán, mô tả, số ảnh để đánh dấu dữ liệu có đủ thông tin.

#### Bonbanh
- Tiêu đề/giá nằm trong phần `<h1>` và `<b itemprop="price">`, nhưng HTML thường chứa nhiều newline nên cần làm phẳng chuỗi.
- Thông số kỹ thuật ở các `div.row` với `label` thể hiện tên, `span.inp` chứa giá trị.
- Lọc chỉ giữ các keys quan trọng (Năm sản xuất, Tình trạng, Hộp số, v.v.) để dễ chuẩn hóa.

In [ ]:
def extract_chotot_features(url):
    """
    Trích xuất features chi tiết từ trang listing Chotot.
    Trả về dict với các label rõ ràng để merge sau này.
    """
    try:
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
        }
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')
        
        features = {}  
        
        # Tiêu đề
        title_elem = soup.find('h1')
        if title_elem:
            features['title'] = title_elem.get_text(strip=True)
        
        # Giá
        price_elem = soup.find('b', class_='p26z2wb')
        if price_elem:
            features['price'] = price_elem.get_text(strip=True)
        
        # Features từ div.p1ja3eq0 (label trước, value sau)
        feature_divs = soup.find_all('div', class_='p1ja3eq0')
        for div in feature_divs:
            spans = div.find_all('span', class_='bwq0cbs')
            if len(spans) >= 2:
                key = spans[0].get_text(strip=True)
                value = spans[1].get_text(strip=True)
                if key and value:
                    features[key] = value
        
        # Vị trí 
        location_elem = soup.find('span', class_='bwq0cbs', string=re.compile(r'Quận|Phường|Huyện|Tp|Tỉnh'))
        if location_elem:
            features['location'] = location_elem.get_text(strip=True)
        
        # Người bán
        seller_link = soup.find('a', href=re.compile(r'/user/'))
        if seller_link:
            seller_name = seller_link.find('b')
            if seller_name:
                features['seller'] = seller_name.get_text(strip=True)
        
        # Mô tả 
        desc_elem = soup.find('div', class_='des_txt')
        if desc_elem:
            features['description'] = desc_elem.get_text(strip=True)
        
        return features
    except Exception as e:
        print(f"Lỗi khi trích xuất từ {url}: {e}")
        return {}


### Giải thích hàm trích xuất Chotot
Hàm này giả lập một trình duyệt nhẹ, tải về HTML chi tiết của listing, rồi lần lượt gom các cặp nhãn và giá trị từ `div.p1ja3eq0` (div chứa các cặp nhãn và giá trị) vào một dictionary. Những trường quan trọng như vị trí, người bán, mô tả hay số ảnh cũng được giữ lại để dễ đánh giá độ đầy đủ trước khi đẩy toàn bộ sang CSV.

#### Bonbanh
- Tiêu đề xuất hiện trong `<h1>`.
- Giá, tên hãng, tên xe nằm trong tiêu đề.
- Các thông số kỹ thuật được phân thành `div.row` (các dòng), mỗi cặp `<label>` và `<span class="inp">` tương ứng với tên và giá trị của thuộc tính.
- Lọc chỉ giữ những thuộc tính cốt lõi như Năm sản xuất, Tình trạng, Hộp số... để bảng dữ liệu không bị loãng với thông tin thừa.
- Sau cùng, ghi thêm vị trí và người bán để liên kết ngược tới nguồn.

In [ ]:
def extract_bonbanh_features(url):
    """
    Trích xuất features từ URL Bonbanh.
    """
    try:
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
        }
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')
        
        features = {}
        
        # Tiêu đề
        title_elem = soup.find('h1')
        if title_elem:
            features['title'] = title_elem.get_text(strip=True)
    
        # Các thông số kỹ thuật
        desired_keys = [
            'Năm sản xuất',
            'Tình trạng',
            'Số Km đã đi',
            'Xuất xứ',
            'Kiểu dáng',
            'Hộp số',
            'Động cơ',
            'Màu ngoại thất',
            'Màu nội thất',
            'Số chỗ ngồi',
            'Số cửa',
            'Dẫn động'
        ]
        rows = soup.find_all('div', class_='row')
        for row in rows:
            label_elem = row.find('label')
            inp_elem = row.find('span', class_='inp')
            if label_elem and inp_elem:
                key = label_elem.get_text(strip=True).replace(':', '')
                value = inp_elem.get_text(strip=True)
                if key in desired_keys and value:
                    features[key] = value
        
        # Vị trí + người bán 
        features['location'] = 'Hà Nội'
        seller_elem = soup.find('a', class_='cname')
        if seller_elem:
            features['seller'] = seller_elem.get_text(strip=True)
        
        return features
    except Exception as e:
        print(f"Lỗi khi trích xuất từ {url}: {e}")
        return {}


### Giải thích hàm trích xuất Bonbanh
Chúng ta giữ nguyên tiêu đề như cách người dùng gõ lên trang, chỉ dọn phần head/tail để tránh khoảng trắng dư. Sau khi trang đầy đủ được tải về, danh sách `div.row` cung cấp cặp `label`/`span.inp` nên hàm chỉ nhặt ra những thuộc tính cốt lõi (năm sản xuất, hộp số, màu sắc...) để tránh ghi quá nhiều thông tin thừa. Cuối cùng, location và seller được ghi vào để giữ nguồn gốc dữ liệu.

### 3. Triển khai Thu thập URL
Trước khi trích xuất feature, chúng ta cần danh sách URL cụ thể. Chotot cung cấp API giúp tránh việc iterate qua từng trang tìm kiếm; Bonbanh không có API công khai nên phải scrape danh sách car-item.

#### Thu thập URL từ Chotot và Bonbanh
- Chotot: gọi API `gateway.chotot.com/v1/public/ad-listing` với tham số limit để lấy top listings.
- Bonbanh: tải trang `https://bonbanh.com/oto`, duyệt `li.car-item`, gom tag `<a>` vào danh sách URL.

In [ ]:
def collect_chotot_urls(n=10):
    """
    Thu thập N URL đầu từ trang mua bán ô tô trên chotot.com
    bằng cách gọi API gateway.
    """
    base_url = "https://gateway.chotot.com/v1/public/ad-listing"
    params = {
        'w': 1,          # web flag
        'limit': n,      # số lượng listings
        'st': 's,k',     # sort by score, keyword
        'f': 'p',        # filter published
        'cg': 2010,      # category: mua bán ô tô
        'o': 0,          # offset
        'page': 0        # page
    }
    
    try:
        response = requests.get(base_url, params=params, timeout=10)
        response.raise_for_status()
        data = response.json()
        
        urls = []
        for ad in data.get('ads', []):
            list_id = ad.get('list_id')
            if list_id:
                url = f"https://xe.chotot.com/mua-ban-oto/{list_id}.htm"
                urls.append(url)
        
        return urls[:n]
    except requests.RequestException as e:
        print(f"Lỗi khi gọi API: {e}")
        return []

# Thu thập N URL đầu
chotot_urls = collect_chotot_urls(N)
print(f"Đã thu thập {len(chotot_urls)} URL từ Chotot:")
for url in chotot_urls:
    print(url)

### Giải thích hàm collect_chotot_urls
Ta gọi API `gateway.chotot.com/v1/public/ad-listing` với các tham số lọc chuẩn để lấy những listing mới nhất và dựng lại URL hoàn chỉnh bằng `list_id`. Cách này tránh phải lướt qua từng trang search và giúp lấy nhanh một tập mẫu có thể mở rộng về sau.

In [ ]:
def collect_bonbanh_urls(n=10):
    """
    Thu thập N URL đầu từ trang mua bán ô tô cũ trên bonbanh.com
    bằng cách scrape trang tìm kiếm.
    """
    try:
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
        }
        response = requests.get("https://bonbanh.com/oto", headers=headers, timeout=10)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')
        
        urls = []
        # Tìm car listings trong li.car-item
        listings = soup.find_all('li', class_=lambda x: x and 'car-item' in x)
        for li in listings:
            a_tag = li.find('a', href=True)
            if a_tag:
                href = a_tag['href']
                if href.startswith('/'):
                    full_url = f"https://bonbanh.com{href}"
                else:
                    full_url = f"https://bonbanh.com/{href}"
                if full_url not in urls:
                    urls.append(full_url)
        
        return urls[:n]
    except requests.RequestException as e:
        print(f"Lỗi khi scrape Bonbanh: {e}")
        return []

# Thu thập N URL đầu từ Bonbanh
bonbanh_urls = collect_bonbanh_urls(N)
print(f"Đã thu thập {len(bonbanh_urls)} URL từ Bonbanh:")
for url in bonbanh_urls:
    print(url)

### Giải thích hàm collect_bonbanh_urls
Vì Bonbanh không cung cấp API, hàm tải trang `https://bonbanh.com/oto`, tìm các `li.car-item` và gom liên kết trong mỗi `<a>` để tạo danh sách URL. Mỗi href được chuyển sang dạng tuyệt đối và các entry trùng được loại bỏ, nên nếu cần ta chỉ việc mở rộng bằng cách duyệt thêm các trang `page=N`.

### 4. Triển khai Trích xuất và Lưu Dữ liệu
Dùng danh sách URL thu thập được để thực thi từng hàm trích xuất riêng biệt, gom kết quả thành DataFrame rồi ghi ra CSV. Chuỗi try/except bảo đảm một listing lỗi không dừng toàn bộ quá trình.

In [85]:
# I. Trích xuất dữ liệu từ Chotot
chotot_data = []
for url in chotot_urls:
    try:
        features = extract_chotot_features(url)
        chotot_data.append(features)
    except Exception as e:
        print(f"Lỗi với URL Chotot {url}: {e}")

df_chotot = pd.DataFrame(chotot_data)
df_chotot.to_csv('../datasets/chotot_car_features.csv', index=False)
print(f"Chotot: Đã lưu {len(chotot_data)} bản ghi vào datasets/chotot_car_features.csv")

# II. Trích xuất dữ liệu từ Bonbanh
bonbanh_data = []
for url in bonbanh_urls:
    try:
        features = extract_bonbanh_features(url)
        bonbanh_data.append(features)
    except Exception as e:
        print(f"Lỗi với URL Bonbanh {url}: {e}")

df_bonbanh = pd.DataFrame(bonbanh_data)
df_bonbanh.to_csv('../datasets/bonbanh_car_features.csv', index=False)
print(f"Bonbanh: Đã lưu {len(bonbanh_data)} bản ghi vào datasets/bonbanh_car_features.csv")

Chotot: Đã lưu 10 bản ghi vào datasets/chotot_car_features.csv
Bonbanh: Đã lưu 10 bản ghi vào datasets/bonbanh_car_features.csv
Bonbanh: Đã lưu 10 bản ghi vào datasets/bonbanh_car_features.csv


## Kết quả
- Dữ liệu từ Chotot được lưu ở `datasets/chotot_car_features.csv`, mỗi bản ghi bao gồm giá, vị trí, người bán, mô tả và số ảnh để đánh giá mức độ đầy đủ trước khi làm sạch.
- Dữ liệu từ Bonbanh vào `datasets/bonbanh_car_features.csv`, giữ lại tiêu đề gốc của người bán và những thông số kỹ thuật cốt lõi sau khi lọc để tránh ghi quá nhiều trường không cần thiết.